In [ ]:
!pip install cherche

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cherche-1.0.1-py3-none-any.whl


In [ ]:
from cherche import retrieve

In [ ]:

import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
caminho_query = "/content/gdrive/MyDrive/Colab Notebooks/dados-conle-anonimizado-assunto-notnull (1).csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Base de dados câmara dos deputados

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/proposicao-tema-completo.csv")

In [ ]:
df_assunto= pd.read_csv(caminho_query, encoding='utf-8', delimiter=";")

In [ ]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [ ]:
def verificar(y,top_n):
    for d in top_n:
      if y == df.loc[d["id"],"txtNome"]:
        return 1
    return 0

In [ ]:
def avaliacaoRecall(isPreprocess):

    quant_encontrado=0
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = retriever(tokenized_query3)
        #top_n = [df.loc[d["id"],"name"] for d in top_n_stem_l]

        
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    print("R@20: "+str(recall))


# Word n-gram

In [ ]:
def tokenizer1(doc):
    return doc.split("\n")

## 12- Bigram

In [ ]:
def preprocess(txt):

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    ngram = []
    
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.0


In [ ]:
documentos = None
retriever =None

## 13- Trigram

In [ ]:
def preprocess(txt):

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    ngram = []
    
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + " " + w[1] + " " + w[2]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.0


In [ ]:
documentos = None
retriever =None

## 14- Unigram + Bigram

In [ ]:
def preprocess(txt):
    
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt)
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.0


In [ ]:
documentos = None
retriever =None